## Census Tract-level data

In [1]:
# setup
from census import Census
from us import states
import pandas as pd 
import numpy as np


In [2]:

# Tract-level data
#   can also serve as control totals 

# get HH Size by tract
api_key = "71413a49e8eb8071baacc6279df9403154f4a774"
c = Census(api_key,year=2022) # my API key
tracts = c.acs5.get(('NAME','GEO_ID','B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'), 
                     geo={'for': 'tract:*',
                       'in': 'state:{} county:057'.format(states.WA.fips)})

tracts2 = c.acs5st.get(('GEO_ID','S1901_C01_013E'), 
                     geo={'for': 'tract:*',
                       'in': 'state:{} county:057'.format(states.WA.fips)})

tracts = pd.DataFrame(tracts)
tracts2 = pd.DataFrame(tracts2)

# fix geoid
tracts['GEOID2'] = tracts['GEO_ID'].str.slice(start=9)
tracts2['GEOID2'] = tracts2['GEO_ID'].str.slice(start=9)

tracts = tracts.merge(tracts2, on='GEOID2')


In [7]:
HH4pHH = tracts.apply(lambda row: row['B11016_005E'] + row['B11016_013E'] + row['B11016_006E'] + row['B11016_014E'] + row['B11016_007E'] + row['B11016_015E'] + row['B11016_008E'] + row['B11016_016E'], axis=1)
HH4pPop = tracts.apply(lambda row: ((row['B11016_005E'] + row['B11016_013E']) * 4) + ((row['B11016_006E'] + row['B11016_014E']) * 5) + ((row['B11016_007E'] + row['B11016_015E']) * 6) + ((row['B11016_008E'] + row['B11016_016E']) * 7.5), axis=1)
HH4pAvg = HH4pPop.sum() / HH4pHH.sum()

print("Avg size of 4+ person HHs: ",HH4pAvg)


Avg size of 4+ person HHs:  4.736502989977259


In [ ]:

# combine family and non-family households by size
tracts['HH_Total'] = tracts['B11016_001E']
tracts['HH1'] = tracts['B11016_010E']
tracts['HH2'] = tracts.apply(lambda row: row['B11016_003E'] + row['B11016_011E'], axis=1)
tracts['HH3'] = tracts.apply(lambda row: row['B11016_004E'] + row['B11016_012E'], axis=1)
tracts['HH4plus'] = tracts.apply(lambda row: row['B11016_005E'] + row['B11016_006E'] + row['B11016_007E'] + row['B11016_008E'] + row['B11016_013E'] + row['B11016_014E'] + row['B11016_015E'] + row['B11016_016E'], axis=1)
tracts['HH_AvgSize'] = tracts['B25010_001E']
tracts['Wrk_Total'] = tracts['B08006_001E']
tracts['Wrk_Average'] = tracts.apply(lambda row: 0 if row['HH_Total'] == 0 else row['B08006_001E'] / row['HH_Total'], axis=1) # total workers / total households 
tracts['Inc_Median'] = tracts['B19326_001E']
tracts['Inc_Mean'] = tracts['S1901_C01_013E']

# For HH submodel size and income joint distribution
# A two-dimensional distribution table for household size and income was generated using 2017-2021 5-Year ACS data (see Table 19).
# TABLE 19: TWO DIMENSIONAL 2022 HOUSEHOLD SIZE & HOUSEHOLD INCOME DISTRIBUTION
# HOUSEHOLD SIZE x Income level: <25K, 25K≤INCOME<50K, 50K≤INCOME<75K, 75K≤INCOME
# B19001_002E	Less than $10,000  cat 1
# B19001_003E	$10,000 to $14,999 cat 1
# B19001_004E	$15,000 to $19,999 cat 1
# B19001_005E	$20,000 to $24,999 cat 1
# B19001_006E	$25,000 to $29,999 cat 2
# B19001_007E	$30,000 to $34,999 cat 2
# B19001_008E	$35,000 to $39,999 cat 2
# B19001_009E	$40,000 to $44,999 cat 2
# B19001_010E	$45,000 to $49,999 cat 2
# B19001_011E	$50,000 to $59,999 cat 3
# B19001_012E	$60,000 to $74,999 cat 3
# B19001_013E	$75,000 to $99,999 cat 4
# B19001_014E	$100,000 to $124,999 cat 4
# B19001_015E	$125,000 to $149,999 cat 4
# B19001_016E	$150,000 to $199,999 cat 4
# B19001_017E	$200,000 or more cat 4

tracts['INC1'] = tracts.apply(lambda row: row['B19001_002E'] + row['B19001_003E'] + row['B19001_004E'] + row['B19001_005E'], axis=1)                      # < 25k
tracts['INC2'] = tracts.apply(lambda row: row['B19001_006E'] + row['B19001_007E'] + row['B19001_008E'] + row['B19001_009E'] + row['B19001_010E'], axis=1) # 25k-50K
tracts['INC3'] = tracts.apply(lambda row: row['B19001_011E'] + row['B19001_012E'], axis=1)                                                                # 50k-75K
tracts['INC4'] = tracts.apply(lambda row: row['B19001_013E'] + row['B19001_014E'] + row['B19001_015E'] + row['B19001_016E'] + row['B19001_017E'], axis=1) # >75K

# proportions
tracts['HH_Size1'] = tracts['HH1'] / tracts['HH_Total']
tracts['HH_Size2'] = tracts['HH2'] / tracts['HH_Total']
tracts['HH_Size3'] = tracts['HH3'] / tracts['HH_Total']
tracts['HH_Size4plus'] = tracts['HH4plus'] / tracts['HH_Total']

tracts = tracts.drop(columns=['B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                              'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'])

#tracts.to_csv('tracts.csv', index=False)



In [34]:
tracts['HH_AvgSize2'] = round(tracts['HH_AvgSize'],1)
# size_pivot = tracts.pivot_table(index='HH_AvgSize2',columns=['HH1','HH2','HH3','HH4plus'],aggfunc='sum')

size_pivot = tracts.groupby('HH_AvgSize2')[['HH1','HH2','HH3','HH4plus']].agg('sum')
size_pivot.to_csv('HHSize_pivot.csv')


In [ ]:

# blockgroup-level data

# get HH Size by BG
bgs = c.acs5.get(('NAME','GEO_ID','B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'), 
                     geo={'for': 'block group:*',
                       'in': 'state:{} county:057'.format(states.WA.fips)})

bgs = pd.DataFrame(bgs)

# fix geoid
bgs['GEOID2'] = bgs['GEO_ID'].str.slice(start=9)

# combine family and non-family households by size
bgs['HH_Total'] = bgs['B11016_001E']
bgs['HH1'] = bgs['B11016_010E']
bgs['HH2'] = bgs.apply(lambda row: row['B11016_003E'] + row['B11016_011E'], axis=1)
bgs['HH3'] = bgs.apply(lambda row: row['B11016_004E'] + row['B11016_012E'], axis=1)
bgs['HH4plus'] = bgs.apply(lambda row: row['B11016_005E'] + row['B11016_006E'] + row['B11016_007E'] + row['B11016_008E'] + row['B11016_013E'] + row['B11016_014E'] + row['B11016_015E'] + row['B11016_016E'], axis=1)
bgs['HH_AvgSize'] = bgs['B25010_001E']
# bgs['Wrk_Total'] = bgs['B08006_001E']
# bgs['Wrk_Average'] = bgs.apply(lambda row: 0 if row['HH_Total'] == 0 else row['B08006_001E'] / row['HH_Total'], axis=1) # total workers / total households 
bgs['Inc_Median'] = bgs['B19326_001E']



# For HH submodel size and income joint distribution
# A two-dimensional distribution table for household size and income was generated using 2017-2021 5-Year ACS data (see Table 19).
# TABLE 19: TWO DIMENSIONAL 2022 HOUSEHOLD SIZE & HOUSEHOLD INCOME DISTRIBUTION
# HOUSEHOLD SIZE x Income level: <25K, 25K≤INCOME<50K, 50K≤INCOME<75K, 75K≤INCOME
# B19001_002E	Less than $10,000  cat 1
# B19001_003E	$10,000 to $14,999 cat 1
# B19001_004E	$15,000 to $19,999 cat 1
# B19001_005E	$20,000 to $24,999 cat 1
# B19001_006E	$25,000 to $29,999 cat 2
# B19001_007E	$30,000 to $34,999 cat 2
# B19001_008E	$35,000 to $39,999 cat 2
# B19001_009E	$40,000 to $44,999 cat 2
# B19001_010E	$45,000 to $49,999 cat 2
# B19001_011E	$50,000 to $59,999 cat 3
# B19001_012E	$60,000 to $74,999 cat 3
# B19001_013E	$75,000 to $99,999 cat 4
# B19001_014E	$100,000 to $124,999 cat 4
# B19001_015E	$125,000 to $149,999 cat 4
# B19001_016E	$150,000 to $199,999 cat 4
# B19001_017E	$200,000 or more cat 4

bgs['INC1'] = bgs.apply(lambda row: row['B19001_002E'] + row['B19001_003E'] + row['B19001_004E'] + row['B19001_005E'], axis=1)                      # < 25k
bgs['INC2'] = bgs.apply(lambda row: row['B19001_006E'] + row['B19001_007E'] + row['B19001_008E'] + row['B19001_009E'] + row['B19001_010E'], axis=1) # 25k-50K
bgs['INC3'] = bgs.apply(lambda row: row['B19001_011E'] + row['B19001_012E'], axis=1)                                                                # 50k-75K
bgs['INC4'] = bgs.apply(lambda row: row['B19001_013E'] + row['B19001_014E'] + row['B19001_015E'] + row['B19001_016E'] + row['B19001_017E'], axis=1) # >75K

# proportions
bgs['HH_Size1'] = bgs['HH1'] / bgs['HH_Total']
bgs['HH_Size2'] = bgs['HH2'] / bgs['HH_Total']
bgs['HH_Size3'] = bgs['HH3'] / bgs['HH_Total']
bgs['HH_Size4plus'] = bgs['HH4plus'] / bgs['HH_Total']

bgs = bgs.drop(columns=['B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                              'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'])

bgs.to_csv('blockgroups.csv', index=False)

In [35]:
bgs['HH_AvgSize2'] = round(bgs['HH_AvgSize'],1)
# size_pivot = tracts.pivot_table(index='HH_AvgSize2',columns=['HH1','HH2','HH3','HH4plus'],aggfunc='sum')

size_pivot = bgs.groupby('HH_AvgSize2')[['HH1','HH2','HH3','HH4plus']].agg('sum')
size_pivot.to_csv('HHSize_pivot_bg.csv')


In [5]:
# HH Size in adjacent counties

# get HH Size by BG
bgs_island = c.acs5.get(('NAME','GEO_ID','B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'), 
                     geo={'for': 'block group:*',
                       'in': 'state:{} county:029'.format(states.WA.fips)})

bgs_whatcom = c.acs5.get(('NAME','GEO_ID','B11016_001E','B11016_003E','B11016_004E','B11016_005E','B11016_006E','B11016_007E','B11016_008E','B11016_010E','B11016_011E','B11016_012E','B11016_013E','B11016_014E','B11016_015E','B11016_016E','B25010_001E','B01001_001E','B08006_001E','B19326_001E',
                    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E','B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E','B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E','B19001_017E'), 
                     geo={'for': 'block group:*',
                       'in': 'state:{} county:073'.format(states.WA.fips)})

bgs1 = pd.DataFrame(bgs_island)
bgs2 = pd.DataFrame(bgs_whatcom)
bgs = pd.concat([bgs1,bgs2])

# fix geoid
bgs['GEOID2'] = bgs['GEO_ID'].str.slice(start=9)

# combine family and non-family households by size
bgs['HH_Total'] = bgs['B11016_001E']
bgs['HH1'] = bgs['B11016_010E']
bgs['HH2'] = bgs.apply(lambda row: row['B11016_003E'] + row['B11016_011E'], axis=1)
bgs['HH3'] = bgs.apply(lambda row: row['B11016_004E'] + row['B11016_012E'], axis=1)
bgs['HH4plus'] = bgs.apply(lambda row: row['B11016_005E'] + row['B11016_006E'] + row['B11016_007E'] + row['B11016_008E'] + row['B11016_013E'] + row['B11016_014E'] + row['B11016_015E'] + row['B11016_016E'], axis=1)
bgs['HH_AvgSize'] = bgs['B25010_001E']

bgs['HH_AvgSize2'] = round(bgs['HH_AvgSize'],1)
# size_pivot = tracts.pivot_table(index='HH_AvgSize2',columns=['HH1','HH2','HH3','HH4plus'],aggfunc='sum')

size_pivot = bgs.groupby('HH_AvgSize2')[['HH1','HH2','HH3','HH4plus']].agg('sum')
size_pivot.to_csv('HHSize_added_pivot_bg.csv')


## HH Income Index (by tract)

In [30]:
area_income = (tracts['Inc_Mean'] * tracts['HH_Total']).sum() # 5,262,863,780.0
area_avg_income = area_income / (tracts['HH_Total'].sum()) # dived by 50,824 HH = 103,550
tracts['INC_INDEX'] = tracts['Inc_Mean'] / area_avg_income
tracts['INC_INDEX2'] = round(tracts['INC_INDEX'],1)

income_pivot = tracts.groupby('INC_INDEX2')[['INC1','INC2','INC3','INC4']].agg('sum')
income_pivot.to_csv('HHIncome_pivot.csv')

## PUMS HH Size x Income crosstab

In [22]:
# PUMS 2018-22 5 year household data
pums = pd.read_csv('data/PUMS/psam_h53.csv')
pums.head()

,RT,SERIALNO,DIVISION,PUMA10,PUMA20,REGION,ST,ADJHSG,ADJINC,WGTP,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
0,H,2018GQ0000023,9,10100,-9,4,53,1169060,1184371,0,...,0,0,0,0,0,0,0,0,0,0
1,H,2018GQ0000078,9,11502,-9,4,53,1169060,1184371,0,...,0,0,0,0,0,0,0,0,0,0
2,H,2018GQ0000117,9,11900,-9,4,53,1169060,1184371,0,...,0,0,0,0,0,0,0,0,0,0
3,H,2018GQ0000163,9,11401,-9,4,53,1169060,1184371,0,...,0,0,0,0,0,0,0,0,0,0
4,H,2018GQ0000249,9,10901,-9,4,53,1169060,1184371,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
pums['NP'].value_counts()

NP
2     56386
1     55637
3     21608
4     17987
0      8699
5      7746
6      3064
7      1218
8       494
9       192
10      113
11       42
12       35
13        9
20        6
16        3
14        3
15        3
17        1
Name: count, dtype: int64

In [24]:
# recode Household size to categories

size_cat = [
    (pums['NP'] == 1),
    (pums['NP'] == 2),
    (pums['NP'] == 3),
    (pums['NP'] >= 4)
]
size_cat_labels = ['SIZE1','SIZE2','SIZE3','SIZE4']
pums['HHSIZE'] = np.select(size_cat, size_cat_labels, default='NA')

pums['HHSIZE'].value_counts() # freq table

HHSIZE
SIZE2    56386
SIZE1    55637
SIZE4    30916
SIZE3    21608
NA        8699
Name: count, dtype: int64

In [11]:
# recode income to categories
pums['HHINC_22'] = pums['HINCP'] * 1.042311 # 2022 ADJINC factor

income_cat = [
    (pums['HHINC_22'] < 25000),
    (pums['HHINC_22'] >= 25000) & (pums['HHINC_22'] < 50000),
    (pums['HHINC_22'] >= 50000) & (pums['HHINC_22'] < 75000),
    (pums['HHINC_22'] >= 75000)
]
income_cat_labels = ['INC1','INC2','INC3','INC4']
pums['INC_CAT'] = np.select(income_cat, income_cat_labels, default='NA')

pums['INC_CAT'].value_counts() # freq table

INC_CAT
INC4    82446
NA      25927
INC3    23390
INC2    23352
INC1    18131
Name: count, dtype: int64

In [17]:
pums_sizebyinc = pd.crosstab(pums['HHSIZE'],pums['INC_CAT'], rownames=['HHSize'],colnames=['HHIncome'])
pums_sizebyinc.to_csv('PUMS_SizebyIncome.csv')

In [8]:
# PUMS 2018-22 5 year person data
pumsp = pd.read_csv('data/PUMS/psam_p53.csv')

In [25]:
# recode workers to cat
# ESR = Employment Status Recode
# from MCCOG: ESR %in% c(1,2,4,5) ~ 1, # Employed or Armed Forces
#             ESR %in% c(3,6) | is.na(ESR) ~ 0 # # Unemployed, Not in Labor Force, or under 16

pumsp['Worker'] = np.where(pumsp['ESR'].isin([1,2,4,5]),1,0)

# Group by serial no, join to HH records
pums_workers = pumsp.groupby(['SERIALNO'])['Worker'].sum()


In [26]:
pums2 = pums.merge(pums_workers, how="left", on="SERIALNO")

In [27]:
pums2.columns


Index(['RT', 'SERIALNO', 'DIVISION', 'PUMA10', 'PUMA20', 'REGION', 'ST',
       'ADJHSG', 'ADJINC', 'WGTP',
       ...
       'WGTP73', 'WGTP74', 'WGTP75', 'WGTP76', 'WGTP77', 'WGTP78', 'WGTP79',
       'WGTP80', 'HHSIZE', 'Worker'],
      dtype='object', length=240)

In [28]:

workers_cat = [
    (pums2['Worker'] == 0),
    (pums2['Worker'] == 1),
    (pums2['Worker'] == 2),
    (pums2['Worker'] >= 3)
]
workers_cat_labels = ['WRK0','WRK1','WRK2','WRK3']
pums2['WRK_CAT'] = np.select(workers_cat, workers_cat_labels, default='NA')

pums_sizebywrk = pd.crosstab(pums2['HHSIZE'],pums2['WRK_CAT'], rownames=['HHSize'],colnames=['HHWorkers'])
pums_sizebywrk.to_csv('PUMS_SizebyWorkers.csv')

In [16]:
from itertools import product

no = [1,2,3,4,5,6,7,8,9,10]
hhsize = [1,2,3,4] 
hhinc = [1,2,3,4]
hhwrk = [0,1,2,3]
tothh = [0]
combinations = list(product(no,hhsize,hhinc,hhwrk,tothh))
test = pd.DataFrame(combinations, columns=['NO','HHSIZE','HHINC','HHWRK','TOTHH'])
test.head()

,NO,HHSIZE,HHINC,HHWRK,TOTHH
0,1,1,1,0,0
1,1,1,1,1,0
2,1,1,1,2,0
3,1,1,1,3,0
4,1,1,2,0,0
